# ¡Día de Gradio!

Hoy crearemos interfaces de usuario utilizando el increíblemente simple marco Gradio.

## ¡Prepárate para la alegría!

Ten en cuenta que las pantallas de Gradio pueden aparecer en "modo oscuro" o "modo claro" según la configuración de tu computadora.

In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [2]:
import gradio as gr # oh yeah!

In [3]:
# Cargar variables de entorno en un archivo llamado .env
# Imprimir los prefijos de clave para facilitar la depuración

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

OpenAI API Key existe y empieza por sk-proj-
Anthropic API Key existe y empieza por sk-ant-
Google API Key existe y empieza por AIzaSyBr


In [4]:
# Conéctese a OpenAI, Anthropic y Google; comente las líneas de Claude o Google si no las está usando

openai = OpenAI()

#claude = anthropic.Anthropic()

# google.generativeai.configure()

In [5]:
# Un mensaje genérico del sistema: ¡no más IA adversarias sarcásticas!

system_message = "Eres un asistente útil"

In [6]:
# Envolvamos una llamada a GPT-4o-mini en una función simple

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("¿Cual es la fecha de hoy?")

## Momento de crear nuestra interfaz de usuario

In [ ]:
# Aquí hay una función simple

def shout(text):
    print(f"Has llamado a la función shout con el mensaje: {text}")
    return text.upper()

In [ ]:
shout("Hola")

In [ ]:
# La simplicidad de Gradio. Esto puede aparecer en "modo claro". Más adelante te mostraré cómo hacerlo en modo oscuro.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Agregar share=True significa que se puede acceder a él de forma pública
# Hay disponible un alojamiento más permanente mediante una plataforma llamada Spaces de HuggingFace,
# que abordaremos la próxima semana

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Agregar inbrowser=True abre una nueva ventana del navegador automáticamente

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Forzar el modo oscuro

Gradio aparece en modo claro o en modo oscuro según la configuración del navegador y la computadora. Hay una manera de forzar que Gradio aparezca en modo oscuro, pero Gradio recomienda no hacerlo ya que debería ser una preferencia del usuario (particularmente por razones de accesibilidad). Pero si desea forzar el modo oscuro para sus pantallas, a continuación se muestra cómo hacerlo.

In [ ]:
# Define esta variable y luego pasa js=force_dark_mode al crear la interfaz

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Entradas y Salidas

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Tu Mensaje:", lines=6)],
    outputs=[gr.Textbox(label="Respuesta:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Y ahora, cambiando la función de "shout" a "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Tu Mensaje:", lines=6)],
    outputs=[gr.Textbox(label="Respuesta:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Usemos Markdown
# ¿Te preguntas por qué es importante configurar system_message cuando no se hace referencia a él en el código que se encuentra debajo?
# Estoy aprovechando que system_message es una variable global, que se usa en la función message_gpt (échale un vistazo)
# No es una gran práctica de ingeniería de software, pero es bastante común durante la investigación y el desarrollo de Jupyter Lab.

system_message = "Eres una asistente útil que responde en formato markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Tu mensaje:")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Vamos a crear una llamada que transmita resultados
# Si deseas un repaso de los generadores (la palabra clave "yield"),
# Echa un vistazo al cuaderno de Python intermedio en la carpeta de la semana 1.


def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Tu :")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Tu Mensaje:")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

## Una mejora menor

He realizado una pequeña mejora en este código.

Anteriormente, tenía estas líneas:

```
for chunk in result:
yield chunk
```

En realidad, hay una forma más elegante de lograr esto (que los expertos en Python podrían llamar más "Pythonico"):

`yield from result`

Abordo esto con más detalle en el cuaderno de Python intermedio en la carpeta week1; échale un vistazo si quieres más información.

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Tu mensaje:"), gr.Dropdown(["GPT", "Claude"], label="Selecciona un modelo:", value="GPT")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

# Cómo crear un generador de folletos de empresa

Ahora ya sabes cómo hacerlo: ¡es muy sencillo!

<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../important.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#900;">Antes de leer las próximas celdas</h2>
<span style="color:#900;">
Intente hacerlo usted mismo: vuelva al folleto de la empresa en la semana 1, día 5 y agregue una interfaz de usuario de Gradio al final. Luego, venga y observe la solución.
</span>
</td>
</tr>
</table>

In [5]:
# Una clase para representar una página web

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No se ha encontrado título de la página"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [6]:
# ¡Muchas gracias a Bill G., que se dio cuenta de que una versión anterior de este programa tenía un error! Ya está corregido.

system_message = "Eres un asistente que analiza el contenido de la página de inicio \
del sitio web de una empresa y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados.\
Responde en formato Markdown."

In [7]:
def stream_brochure(company_name, url, model):
    prompt = f"Genera un folleto de la empresa {company_name}. Esta es su página de destino:\n"    
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [8]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Nombre de la Empresa:"),
        gr.Textbox(label="Landing page, recuerda incluir http:// o https://"),
        gr.Dropdown(["GPT", "Claude"], label="Selecciona un modelo")],
    outputs=[gr.Markdown(label="Folleto:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [9]:
import google.generativeai as genai
import os

genai.configure(api_key=google_api_key)

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("The opposite of hot is")
print(response.text)

cold

